In [1]:
pip install bcrypt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 593.7/593.7 kB 4.8 MB/s eta 0:00:00


In [2]:
from google.colab import files
uploaded = files.upload()

Saving common_passwords.csv to common_passwords.csv


In [3]:
import pandas as pd
import io

# Read the uploaded CSV file
for uploaded_filename, uploaded_content in uploaded.items():
    df = pd.read_csv(io.StringIO(uploaded_content.decode('utf-8')))
    break  # Assuming there is only one uploaded file

# Display the first 10 lines of the DataFrame
df.head(10)


,password,length,num_chars,num_digits,num_upper,num_lower,num_special,num_vowels,num_syllables
0,123456,6,0,6,0,0,0,0,1
1,password,8,8,0,0,8,0,2,2
2,12345678,8,0,8,0,0,0,0,1
3,qwerty,6,6,0,0,6,0,1,3
4,123456789,9,0,9,0,0,0,0,1
5,12345,5,0,5,0,0,0,0,1
6,1234,4,0,4,0,0,0,0,1
7,111111,6,0,6,0,0,0,0,1
8,1234567,7,0,7,0,0,0,0,1
9,dragon,6,6,0,0,6,0,2,2


In [ ]:
pip install pycryptodome

In [4]:
from cryptography.hazmat.primitives import hashes, hmac, serialization
from cryptography.hazmat.primitives.asymmetric import rsa, padding, ec
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
from cryptography.exceptions import InvalidTag
import pandas as pd
import secrets


In [5]:
# Function to generate a hash of plaintext using SHA-256
def generate_hash(plaintext):
    digest = hashes.Hash(hashes.SHA256(), backend=default_backend())
    digest.update(plaintext)
    hashed_plaintext = digest.finalize()
    return hashed_plaintext

def aes_gcm_encrypt(plaintext, key):
    iv = secrets.token_bytes(12)  # Initialization Vector (IV) for AES-GCM (12 bytes)
    cipher = Cipher(algorithms.AES(key), modes.GCM(iv), backend=default_backend())
    encryptor = cipher.encryptor()
    encryptor.authenticate_additional_data(b'authenticated but not encrypted data')
    ciphertext = encryptor.update(plaintext) + encryptor.finalize()
    tag = encryptor.tag
    return ciphertext, tag, iv

def aes_gcm_decrypt(ciphertext, key, tag, iv):
    cipher = Cipher(algorithms.AES(key), modes.GCM(iv, tag), backend=default_backend())
    decryptor = cipher.decryptor()
    decryptor.authenticate_additional_data(b'authenticated but not encrypted data')
    plaintext = decryptor.update(ciphertext) + decryptor.finalize()
    return plaintext

def compute_hmac(key, plaintext):
    h = hmac.HMAC(key, hashes.SHA256(), backend=default_backend())
    h.update(plaintext)
    hmac_digest = h.finalize()
    return hmac_digest

def rsa_encrypt(plaintext, public_key):
    ciphertext = public_key.encrypt(
        plaintext,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    return ciphertext

def rsa_decrypt(ciphertext, private_key):
    plaintext = private_key.decrypt(
        ciphertext,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    return plaintext

# Example code to iterate through each row in the DataFrame
for index, row in df.iterrows():
    plaintext = row.to_csv(index=False).encode('utf-8')

    # Generate a random key of 16 bytes (128 bits)
    random_key = secrets.token_bytes(16)

    # Generate hash of plaintext
    hashed_plaintext = generate_hash(plaintext)

    # Perform AES-GCM encryption
    ciphertext, tag, iv = aes_gcm_encrypt(plaintext, random_key)

    # Compute HMAC digest
    hmac_digest = compute_hmac(random_key, plaintext)

    # Generate RSA key pair
    private_key = rsa.generate_private_key(public_exponent=65537, key_size=2048, backend=default_backend())
    public_key = private_key.public_key()

    # Encrypt AES-GCM key with RSA public key
    encrypted_key = rsa_encrypt(random_key, public_key)

    # Decrypt AES-GCM key using RSA private key
    decrypted_key = rsa_decrypt(encrypted_key, private_key)

    # Use decrypted AES-GCM key to perform decryption
    decrypted_plaintext = aes_gcm_decrypt(ciphertext, decrypted_key, tag, iv)

    # Your further processing with decrypted plaintext...

     # Split the encrypted data by lines and print each line
    print(f"Row {index + 1} - Plaintext Message:")
    for line in plaintext.decode().splitlines():
        print(line)

    print(f"Row {index + 1} - Hashed Plaintext:")
    for line in hashed_plaintext.hex().splitlines():
        print(line)

    print(f"Row {index + 1} - AES-GCM Ciphertext:")
    for line in ciphertext.hex().splitlines():
        print(line)

    print(f"Row {index + 1} - AES-GCM Tag:")
    for line in tag.hex().splitlines():
        print(line)

    print(f"Row {index + 1} - HMAC Digest:")
    for line in hmac_digest.hex().splitlines():
        print(line)

    # Printing for demonstration purposes
    print(f"Row {index + 1} - Decrypted Plaintext:")
    print(decrypted_plaintext.decode('utf-8'))
    print()


Row 1 - Plaintext Message:
0
123456
6
0
6
0
0
0
0
1
Row 1 - Hashed Plaintext:
b947dab7b39447817d3061df3bdb028c2b5d689105a079ca23e40bc97df31e3c
Row 1 - AES-GCM Ciphertext:
c930ac751c0b14963437dbbb8ebd8fe038fb418d5f2cff84a4
Row 1 - AES-GCM Tag:
3874e628d1e8f98a7398477c72a0110a
Row 1 - HMAC Digest:
969bfa47fdc58a7fe40aa991e08cec1a701338b0866476c7750495a5976270d5
Row 1 - Decrypted Plaintext:
0
123456
6
0
6
0
0
0
0
1


Row 2 - Plaintext Message:
1
password
8
8
0
0
8
0
2
2
Row 2 - Hashed Plaintext:
54f8fcb667d46760f8df985bd4700d3e43d2fe12fe7c17b65ddbe1abeb950fbf
Row 2 - AES-GCM Ciphertext:
2539ce042a93785ec348933bd73598ee3eb76519b6e805cf376d27
Row 2 - AES-GCM Tag:
253a3ba281e56dc6302ec3144b9bc024
Row 2 - HMAC Digest:
a2cf9e5afc6db8629543d83929417f80299f8d77740a07b460d67f3059ca99d9
Row 2 - Decrypted Plaintext:
1
password
8
8
0
0
8
0
2
2


Row 3 - Plaintext Message:
2
12345678
8
0
8
0
0
0
0
1
Row 3 - Hashed Plaintext:
248811f33b15246494dfbb42141ba8d514d99ca70709c80164884f105f0d1f15
Row 3 - AES

KeyboardInterrupt: ignored

AES-GCM+ECDSA+EEC

In [6]:
from cryptography.hazmat.primitives.asymmetric import ec, utils
from cryptography.hazmat.primitives import serialization, hashes
import hashlib

from cryptography.exceptions import InvalidSignature



# Function to generate a digital signature
def generate_signature(private_key, plaintext):
    signature = private_key.sign(
        plaintext,
        ec.ECDSA(hashes.SHA256())
    )
    return signature

# Function to verify a digital signature
def verify_signature(public_key, plaintext, signature):
    try:
        public_key.verify(
            signature,
            plaintext,
            ec.ECDSA(hashes.SHA256())
        )
        return True  # Signature is valid
    except InvalidSignature:
        return False  # Signature is invalid

# Function to generate a hash of plaintext using SHA-256
def generate_hash(plaintext):
    digest = hashes.Hash(hashes.SHA256(), backend=default_backend())
    digest.update(plaintext)
    hashed_plaintext = digest.finalize()
    return hashed_plaintext

# Function to generate a unique IV for AES-GCM encryption
def generate_unique_iv():
    return secrets.token_bytes(12)  # Generating a 12-byte (96-bit) random IV

def aes_gcm_encrypt(plaintext, key):
    iv = generate_unique_iv()  # Generating a new IV for each encryption
    cipher = Cipher(algorithms.AES(key), modes.GCM(iv), backend=default_backend())
    encryptor = cipher.encryptor()
    encryptor.authenticate_additional_data(b'authenticated but not encrypted data')
    ciphertext = encryptor.update(plaintext) + encryptor.finalize()
    tag = encryptor.tag[:16]  # Ensure the authentication tag length is 16 bytes (128 bits)
    return ciphertext, tag, iv

def aes_gcm_decrypt(ciphertext, key, tag, iv):
    cipher = Cipher(algorithms.AES(key), modes.GCM(iv, tag), backend=default_backend())
    decryptor = cipher.decryptor()
    decryptor.authenticate_additional_data(b'authenticated but not encrypted data')
    plaintext = decryptor.update(ciphertext) + decryptor.finalize()
    return plaintext

# Function to compute HMAC digest
def compute_hmac(key, plaintext):
    h = hmac.HMAC(key, hashes.SHA256(), backend=default_backend())
    h.update(plaintext)
    hmac_digest = h.finalize()
    return hmac_digest

# Function to generate ECC key pair
def generate_ecc_key_pair():
    private_key = ec.generate_private_key(ec.SECP256R1(), default_backend())
    public_key = private_key.public_key()
    private_pem = private_key.private_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PrivateFormat.TraditionalOpenSSL,
        encryption_algorithm=serialization.NoEncryption()
    )
    public_pem = public_key.public_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PublicFormat.SubjectPublicKeyInfo
    )
    return private_key, public_key

def ecc_key_agreement(private_key, public_key):
    shared_key = private_key.exchange(ec.ECDH(), public_key)
    return shared_key  # The shared key for symmetric encryption

def ecc_encrypt(shared_key, plaintext):
    iv = secrets.token_bytes(12)  # Initialization Vector (IV) for AES-GCM (12 bytes)
    cipher = Cipher(algorithms.AES(shared_key), modes.GCM(iv), backend=default_backend())
    encryptor = cipher.encryptor()
    encryptor.authenticate_additional_data(b'authenticated but not encrypted data')
    ciphertext = encryptor.update(plaintext) + encryptor.finalize()
    tag = encryptor.tag
    tag = tag[:16]  # Ensure the authentication tag length is 16 bytes (128 bits)
    return ciphertext, tag, iv

def ecc_decrypt(shared_key, ciphertext, tag, iv):
    cipher = Cipher(algorithms.AES(shared_key), modes.GCM(iv, tag), backend=default_backend())
    decryptor = cipher.decryptor()
    decryptor.authenticate_additional_data(b'authenticated but not encrypted data')
    plaintext = decryptor.update(ciphertext) + decryptor.finalize()
    return plaintext

# Iterate through each row in the DataFrame
for index, row in df.iterrows():
    plaintext = row.to_csv(index=False).encode('utf-8')

    # Generate a random key of 16 bytes (128 bits)
    random_key = secrets.token_bytes(16)
    print("Random Key:", random_key.hex())

    # Generate hash of plaintext
    hashed_plaintext = generate_hash(plaintext)

    # Perform AES-GCM encryption
    ciphertext, tag, iv = aes_gcm_encrypt(plaintext, random_key)

    # Compute HMAC digest
    hmac_digest = compute_hmac(random_key, plaintext)

    # Generate ECC key pair
    private_key_ecc, public_key_ecc = generate_ecc_key_pair()

    # Perform ECC key agreement
    shared_key = ecc_key_agreement(private_key_ecc, public_key_ecc)

    # Perform ECC-based symmetric encryption
    ecc_ciphertext, ecc_tag, ecc_iv = ecc_encrypt(shared_key, plaintext)

    # Perform ECC-based symmetric decryption
    ecc_decrypted_data = ecc_decrypt(shared_key, ecc_ciphertext, ecc_tag, ecc_iv)

    # Generate digital signature
    signature = generate_signature(private_key_ecc, plaintext)

    # Verify digital signature
    signature_verified = verify_signature(public_key_ecc, plaintext, signature)

    # Split the encrypted data by lines and print each line
    print(f"Row {index + 1} - Plaintext Message:")
    for line in plaintext.decode().splitlines():
        print(line)

    print(f"Row {index + 1} - Hashed Plaintext:")
    for line in hashed_plaintext.hex().splitlines():
        print(line)

    print(f"Row {index + 1} - AES-GCM Ciphertext:")
    for line in ciphertext.hex().splitlines():
        print(line)

    print(f"Row {index + 1} - AES-GCM Tag:")
    for line in tag.hex().splitlines():
        print(line)

    print(f"Row {index + 1} - HMAC Digest:")
    for line in hmac_digest.hex().splitlines():
        print(line)

    print(f"Row {index + 1} - ECC Encrypted Data:")
    for line in ecc_ciphertext.splitlines():
        print(line)

    print(f"Row {index + 1} - ECC Decrypted Data:")
    for line in ecc_decrypted_data.splitlines():
        print(line)

    print(f"Row {index + 1} - Digital Signature:")
    print(signature.hex())

    print(f"Row {index + 1} - Signature Verification Status:")
    print("Signature is valid." if signature_verified else "Signature is invalid.")

    print()

    print()


Streaming output truncated to the last 5000 lines.
b'8'
b'8'
b'0'
b'0'
b'8'
b'0'
b'3'
b'3'
Row 302 - Digital Signature:
30450220185b7f1bb8fd0c0304cbdcba83c5ee1e05367ef871f9a215526bfca44990fbed022100cdc5eb3e351b508b722bc3746348ccaba58bc5a5dd92051ebf96316e2e3699fa
Row 302 - Signature Verification Status:
Signature is valid.


Random Key: b7cebe0eba95538056795555a1d4513a
Row 303 - Plaintext Message:
302
qazwsxedc
9
9
0
0
9
0
2
1
Row 303 - Hashed Plaintext:
c3bcaf2e52ad92dc1ae680e645013856a6fb14cbbb36ad12480793ceea751a7f
Row 303 - AES-GCM Ciphertext:
587e4ac566b53f754b389ed11a3ceb1c2a6e4bbf7e5cdca503a142c7709d
Row 303 - AES-GCM Tag:
a2060557f82bc80b17e0c6e0b57b7d49
Row 303 - HMAC Digest:
420e75b0437912c6eaf03111ed09586c1e2bd707b1ed1a151b6be822577ac5bd
Row 303 - ECC Encrypted Data:
b'\x99\x9d\x9djr\xa1F\\\x9d\xd6\xef\xa7\x1d\x14'
b'\x89h\xc1\x1b\xea\xfb\x99\xab[\x92\xe8\xc9\x845\x8d'
Row 303 - ECC Decrypted Data:
b'302'
b'qazwsxedc'
b'9'
b'9'
b'0'
b'0'
b'9'
b'0'
b'2'
b'1'
Row 303 - Digital 

KeyboardInterrupt: ignored